In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "./autogen/OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-3.5-turbo"],
    },
)

In [3]:
def use_case1_web_info_task(web_task_query):
    llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    }
    assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,)

    user_proxy = autogen.UserProxyAgent(
        name="web_looker",
        human_input_mode="TERMINATE",
        max_consecutive_auto_reply=5,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        code_execution_config={"work_dir": "web"},
        llm_config=llm_config,
        system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
    )

    # the assistant receives a message from the user, which contains the task description
    user_proxy.initiate_chat(
        assistant,
        message=f"""
    {web_task_query}
    """,
    )

In [5]:
# Successful prompts for the web info task
# 1. Reading through arxiv papers
# use_case1_web_info_task("Who should read this paper: https://arxiv.org/abs/2308.08155")

Exploring the teaching use case

In [13]:
def use_case2_research_workflows(tasks_list):
    
    """
    Creating research workflows from a list of tasks.
    task1 = 'Find arxiv papers that show comparisons betweewn how humans and machines learn the same tasks. '

    user_proxy.initiate_chat(assistant, message=task1)

    task2 = "Analyse the above papers mentioned and find out a list of interesting and original insights about similarities between human and machine learning."

    user_proxy.initiate_chat(assistant, message=task2, clear_history=False)
    """
    
    llm_config = {
    "request_timeout":600,
    "seed":44,
    "config_list":config_list,
    "temperature":0
    }

    assistant = autogen.AssistantAgent(
        name="assistant", 
        llm_config=llm_config,
        is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    )

    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="TERMINATE",
        is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
        max_consecutive_auto_reply=10,
        code_execution_config={"work_dir": "work_dir",
                            "use_docker": False,
                            },
    )
    
    for task in tasks_list:
        user_proxy.initiate_chat(assistant, message=task, clear_history=False)
    
    
    task_recipe_create =  '''Reflect on the sequence and create a recipe containing all the steps 
                                # necessary and name for it. Suggest well-documented, generalized python function(s)
                                #  to perform similar tasks for coding steps in future. Make sure coding steps and 
                                #  non-coding steps are never mixed in one function. In the docstr of the function(s),
                                #  clarify what non-coding steps are needed to use the language skill of the assistant.
                          '''

Re-usable recipes.

Example from the agentchat_teaching.ipynb notebook in the Autogen github repo:

```

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
)

task1 = '''
This recipe is available for you to reuse..

<begin recipe>
**Recipe Name:** Analyzing and Visualizing Application Domains in Arxiv Papers

**Steps:**

1. Collect relevant papers from arxiv using a search query.
2. Analyze the abstracts of the collected papers to identify application domains.
3. Count the number of papers in each application domain.
4. Generate a bar chart of the application domains and the number of papers in each domain.
5. Save the bar chart as an image file.

Here are the well-documented, generalized Python functions to perform the coding steps in the future:

```python
import requests
import feedparser
import matplotlib.pyplot as plt
from typing import List, Dict

def search_arxiv(query: str, max_results: int = 10) -> List[Dict[str, str]]:
    """
    Search arxiv for papers related to a specific query.
    
    :param query: The search query for arxiv papers.
    :param max_results: The maximum number of results to return. Default is 10.
    :return: A list of dictionaries containing the title, link, and summary of each paper.
    """
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query=all:{query}"
    start = 0
    max_results = f"max_results={max_results}"
    url = f"{base_url}{search_query}&start={start}&{max_results}"
    response = requests.get(url)
    feed = feedparser.parse(response.content)
    
    papers = [{"title": entry.title, "link": entry.link, "summary": entry.summary} for entry in feed.entries]
    return papers

def generate_bar_chart(domains: Dict[str, int], output_file: str) -> None:
    """
    Generate a bar chart of application domains and the number of papers in each domain, and save it as an image file.
    
    :param domains: A dictionary containing application domains as keys and the number of papers as values.
    :param output_file: The name of the output image file.
    """
    fig, ax = plt.subplots()
    ax.bar(domains.keys(), domains.values())
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("Application Domains")
    plt.ylabel("Number of Papers")
    plt.title("Number of Papers per Application Domain")

    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
```

**Usage:**

1. Use the `search_arxiv` function to collect relevant papers from arxiv using a search query.
2. Analyze the abstracts of the collected papers using your language skills to identify application domains and count the number of papers in each domain.
3. Use the `generate_bar_chart` function to generate a bar chart of the application domains and the number of papers in each domain, and save it as an image file.

</end recipe>


Here is a new task:
Plot a chart for application domains of GPT models
'''

user_proxy.initiate_chat(assistant, message=task1)


```

In [ ]:
url = "https://lightning.ai/docs"
prompt = f""""""
use_case1_web_info_task(prompt)